# pymysql

-   파이썬에서 MySQL, MariaDB DBMS와 연동하는 다양한 함수를 제공하는 모듈
-   Python [DB API 2.0](http://www.python.org/dev/peps/pep-0249) 표준을 따른다.
-   https://github.com/PyMySQL/PyMySQL/
-   https://pymysql.readthedocs.io/en/latest/


# 설치

-   조건
    -   python version 3.6 이상
    -   mysql version 5.6 이상
-   설치
    -   `pip install pymysql`


In [ ]:
%pip install pymysql

# 기본 작성 절차

1. Database 연결
    ```python
       connection =  pymysql.connect(host:str="DBMS 서버 ip",
                                     port:int=port번호,
                                     user:str="계정명",
                                     password:str="비밀번호",
                                     db:str="연결할데이터베이스이름")
    ```
    - port 번호 기본값: 3306
    - ContextManager 타입으로 with 구문을 이용해 작성하면 close() 작업을 자동으로 처리한다.
2. Connection을 이용해 Cursor 생성
    - Cursor: 연결된 Database에 sql문을 전송하고 select결과 조회 기능을 제공하는 객체
    ```python
        cursor = connection.cursor()
    ```
    - ContextManager 타입으로 with 구문을 이용해 작성하면 close() 작업을 자동으로 처리한다.
3. Cusror를 이용해 SQL문 실행(DB Server로 전송)
    ```python
        cursor.execute("sql문")
    ```
4. Select 결과 조회
    - select 문을 실행한 경우 cursor를 의 fetch 메소드들을 이용해 select 결과를 조회한다.
    ```python
    result = cursor.fetchall()
    ```
5. 연결 닫기
    - cursor, connection 연결을 닫는다.
    - with문을 이용할 수 있다.
    ```python
    cursor.close()
    connection.close()
    ```


# 예제

## 테이블 생성


In [ ]:
create_sql = """
create table customer(
  id  int  auto_increment  primary key,
  name  varchar(20) not null,
  email  varchar(50) not null unique, 
  tall   double,
  birthday  date,
  created_at  datetime  not null
)
"""
# sql 문 마지막에 `;` 은 붙이지 않는다.

In [ ]:
import pymysql

try:
    conn = None  # connection을 저장할 변수
    
    # 1. Database와 연결.
    conn = pymysql.connect(
        host="127.0.0.1",    # DBMS 의 ip(host) : str
        port=3306,           # DBMS의 port 번호: int
        user='playdata',     # username: str
        password="1111",     # password: str
        db="mydb"            #  연결할 Database이름: str
    )  # 연결 성공하면 연결된 DB와 관련 작업할 수있는 기능을 제공하는 Connection객체를 반환
    
    # 2. Connection을 사용해서 Cursor 객체 생성
    #    Cursor: sql 처리를 하는 기능을 제공.( sql 전송하고 처리결과를 받을 때까지 과정을 관리)
    cursor = conn.cursor()
        
    # 3. SQL 문 전송
    cursor.execute("drop table if exists customer")
    cursor.execute(create_sql)
    
finally:
    # 4. 연결닫기(끊기)
    if conn:
        cursor.close() # 4-1. cursor 연결 닫기
        conn.close()   # 4-2. connection 연결 닫기

## 파이썬 타입과 sql 데이터타입 
- str - 문자열타입(char, varchar, text,...)
- int - 정수(tiny int , int, ....)
- float - 부동소수 실수(float, double)
- decimal.Decimal - 고정소수 실수(decimal)
- datetime.date - date
- datetime.time - time
- datetime.datetime - datetime, timestamp

> ### datetime 모듈
> - 파이썬에서 날짜, 시간을 다루는 모듈
> - 날짜 type(class): date
> - 시간 type: time
> - 날짜시간 type: datetime

In [ ]:
from datetime import date, time, datetime
# 실행 시점
now = datetime.now()
print(now)
print(now.year, now.month, now.day, now.hour, now.minute, now.second)
today = date.today()
print(today)

# 특정 시점의 일시
d1 = date(2000, 10, 2)
print(d1)
d2 = datetime(1990, 2, 7, 10, 22, 53)
print(d2)
t = time(17, 22, 33)
print(t)

## DML

### insert


In [ ]:
sql = "insert into customer (name, email, tall, birthday, created_at) values('이순신', 'lee1@naver.com', 185.23, '2000-09-20', now())"

In [ ]:
# with 문을 이용해 connection, cursor 생성: with block을 빠져 나올 때 자동으로 close() 처리한다.
# DML(insert/update/delete) 구문은 처리 후 commit을 실행해야 영구적으로 적용된다.
with pymysql.connect(host="127.0.0.1", port=3306, user="playdata", password="1111", db="mydb") as conn:
    with conn.cursor() as cursor:
        result = cursor.execute(sql) # 반환값: 처리행수(insert/delete/update된 행수, select 조회행수)
        print("처리 행수:", result)
        conn.commit()
   

### Parameterized Query

-   Parameterized Query
    -   SQL 문에서  **값**이 들어가는 자리에 값 대신 `%s` **placeholder**를 사용한뒤 execute()에서 placeholder에 넣을 값을 list나 tuple로 제공한다.
    -   동일한 쿼리문을 값을 바꿔가면서 여러번 실행할 때 유용하다.


In [ ]:
import pymysql

# insert할 값 입력받기
name = input("이름:")
email = input("이메일주소:")
tall = float(input('키:'))

insert_sql = "insert into customer (name, email, tall, birthday, created_at) values (%s, %s, %s, %s, now())"

with pymysql.connect(host="127.0.0.1", port=3306, user='playdata', password='1111', db='mydb') as conn:
    with conn.cursor() as cursor:
        result = cursor.execute(insert_sql, [name, email, tall, "2010-01-01"])
        conn.commit()
        print("처리 행수:", result)

#### Parameterized Query를 이용해 여러 행 insert

- executemany() 사용
    - insert할 값들을 가진 리스트를 넣어 한번에 여러 행을 insert한다.


In [ ]:
from datetime import datetime, date
datas = [
    ["김인영", "abc2@a.com", 165, date(2005, 1, 12), datetime.now()],
    ["오수철", "def2@a.com", 175, date(1995, 12, 20), datetime.now()],
    ["최유명", "ghi2@a.com", 183, date(1978, 10, 28), datetime.now()],
    ["김명수", "jkl@abc.com", 177, date(2000, 2, 12), datetime.now()],
    ["이지영", "mno@abc.com", 163, date(1995, 4, 21), datetime.now()],
    ["박명수", "pqr@abc.com", 185, date(2002, 7, 5), datetime.now()],
]

In [ ]:
# for data in datas:
#     cursor.execute(insert_sql, data)

In [ ]:
insert_sql = "insert into customer(name, email, tall, birthday, created_at) values(%s, %s, %s, %s, %s)"
with pymysql.connect(host="127.0.0.1", port=3306, user='playdata', password='1111', db='mydb') as conn:
    with conn.cursor() as cursor:
        cnt = cursor.executemany(insert_sql, datas)
        conn.commit()

print("insert된 총 행수:", cnt)

### update/delete

-   코딩 절차는 insert 와 동일


In [ ]:
update_sql = "update customer set tall=%s where id=%s"
tall = float(input("변경할 키: "))
cust_id = int(input("변경할 고객 ID: "))

with pymysql.connect(host="127.0.0.1", port=3306, user='playdata', password='1111', db='mydb') as conn:
    with conn.cursor() as cursor:
        result = cursor.execute(update_sql, [tall, cust_id])
        print("처리 행수: ", result)
        conn.commit()

In [ ]:
update_sql = "update customer set email=%s, tall=%s where id=%s"
email = input("변경할 Email주소: ")
tall = float(input("변경할 키: "))
cust_id = int(input("변경할 고객 ID: "))

with pymysql.connect(host="127.0.0.1", port=3306, user='playdata', password='1111', db='mydb') as conn:
    with conn.cursor() as cursor:
        result = cursor.execute(update_sql, [email, tall, cust_id])
        print("처리 행수: ", result)
        conn.commit()

In [ ]:
delete_sql = "delete from customer where tall > %s"

tall = float(input("삭제기준 키:"))

with pymysql.connect(host="127.0.0.1", port=3306, user='playdata', password='1111', db='mydb') as conn:
    with conn.cursor() as cursor:
        result = cursor.execute(delete_sql, [tall])
        print("처리 행수: ", result)
        conn.commit()

## select (DQL - Data Query Language)

-   조회결과 조회
    -   `cursor.execute("select문")` 실행 후 **`cursor`의 결과 조회 메소드(fetch메소드)를**` 이용해 결과를 받는다.
-   fetch메소드(cursor의 메소드들)
    -   **fetchall()**
        -   조회한 모든 행들을 반환
    -   **fetchmany(size=개수)**
        -   전체 조회한 행들 중 지정한 size개수 만큼 반환.
        -   연속적으로 실행하면 다음 size개수 만큼씩 반환한다.
        -   더 이상 조회한 결과가 없으면 빈 튜플을 반환한다.
    -   **fetchone()**
        -   조회결과 중 첫번째 행만 반환
        -   주로 pk 동등 조건으로 조회한 경우 사용


### fetchall()


In [ ]:
sql = "select id, birthday, name, tall from customer" 
with pymysql.connect(host="127.0.0.1", port=3306, user='playdata', password='1111', db='mydb') as conn:
    with conn.cursor() as cursor:
        result = cursor.execute(sql)
        print("조회행수:", result)
        resultset = cursor.fetchall()

In [ ]:
resultset
# tuple(개별행 - tuple(컬럼값들 ))

In [ ]:
resultset[0][1]

### 조회결과를 dictionary로 반환
- pymysql.cursors.DictCursor 사용
    - Connection 생성시 또는 Cursor 생성시 지정한다.
- key: 컬럼명, value: 컬럼값

In [ ]:
sql = "select id, name, tall from customer" 
with pymysql.connect(host="127.0.0.1", port=3306, user='playdata', password='1111', db='mydb') as conn:
    with conn.cursor(pymysql.cursors.DictCursor) as cursor:  
        result = cursor.execute(sql)
        print("조회행수:", result)
        resultset = cursor.fetchall()

In [ ]:
resultset
# list[dictionary{key:컬럼명, value:컬럼값}]

In [ ]:
resultset[1]['name'], resultset[1]['tall']

### fetchone()


In [ ]:
sql = "select * from customer where id = %s" 
with pymysql.connect(host="127.0.0.1", port=3306, user='playdata', password='1111', db='mydb') as conn:
    with conn.cursor() as cursor:
        result = cursor.execute(sql, [5])
        print("조회행수:", result)
        resultset = cursor.fetchone()

In [ ]:
resultset

In [ ]:
if resultset:  # 조회결과가 없으면 None
    print(resultset)
else:
    print("조회결과가 없음.")

### fetchmany()


In [ ]:
sql = "select id, name, birthday from customer" 
with pymysql.connect(host="127.0.0.1", port=3306, user='playdata', password='1111', db='mydb') as conn:
    with conn.cursor() as cursor:
        result = cursor.execute(sql)
        print("조회행수:", result)
        resultset1 = cursor.fetchmany(size=2)  # 처음 두개
        resultset2 = cursor.fetchmany(size=2)  # 다음 두개
        resultset3 = cursor.fetchmany(size=2)  # 다음 두개
        resultset4 = cursor.fetchmany(size=2)  # 다음 두개 (모두 조회시 빈 튜플 반환함)

In [ ]:
print(resultset1)
print(resultset2)
print(resultset3)
print(resultset4)

### select문을 실행한 cursor 는 iterable 타입

-   for in 문에 select query를 실행한 cursor를 사용하면 조회결과를 한 행씩 조회할 수 있다.


In [ ]:
sql = "select * from customer" 
with pymysql.connect(host="127.0.0.1", port=3306, user='playdata', password='1111', db='mydb') as conn:
    with conn.cursor() as cursor:
        result = cursor.execute(sql)
        print("조회행수:", result)
        for id, name, email, tall, birthday, created_at in cursor:
            print(id, name, email, tall, birthday, created_at, sep=" , ")